# Сбор данных о погоде

В этом ноутбуке мы собираем данные с OpenWeatherMap API

In [ ]:
# Импорт необходимых библиотек
import os
from dotenv import load_dotenv
import pandas as pd
from scripts.weather_api import get_weather_data, process_weather_data, save_weather_data
from config import Config

In [ ]:
# Проверка наличия API ключа
load_dotenv()
if not Config.API_KEY:
    raise ValueError("API ключ не найден. Добавьте его в файл .env")

print(f"Начинаем сбор данных для городов: {', '.join(Config.CITIES)}")

In [ ]:
try:
    # Получаем данные для всех городов
    weather_data = save_weather_data()
    print("Данные успешно сохранены!")
except Exception as e:
    print(f"Ошибка при сборе данных: {e}")

In [ ]:
# Проверка собранных данных
try:
    df = pd.read_csv(Config.DATA_RAW_PATH)
    print(f"Успешно загружено {len(df)} записей")
    display(df.head())
except FileNotFoundError:
    print("Файл с данными не найден. Проверьте путь:", Config.DATA_RAW_PATH)
except Exception as e:
    print(f"Ошибка при загрузке данных: {e}")